In [1]:
import pandas as pd
import numpy as np

In [30]:
#screed_fea = pd.read_csv("../../../../extract_CREEDS/ave_dis_DEG_name.csv",index_col=0)

screed_fea = pd.read_csv("../../../../extract_CREEDS/fea_dis_name.csv",index_col=0)
screed_fea[:2]

,ACTB,ACTG1,ADAR,ALDH2,ALDOA,ANP32B,ANXA1,ANXA2,ANXA5,ARF4,...,UBA52,UBB,UBC,UQCRH,UQCRQ,VIM,YBX1,YWHAQ,YWHAZ,ZFP36
nicotine addiction,0.064248,0.057393,0.008424,0.000000,-0.013307,-0.008606,0.000000,0.0,0.000000,0.015622,...,0.076123,-0.015261,0.066425,0.077111,0.012225,0.0,0.104099,-0.022610,0.018012,0.000000
rhinovirus infection,-0.018748,-0.066498,0.013587,0.017732,0.014883,0.000000,-0.024498,0.0,-0.049095,0.000000,...,0.013136,0.000000,0.000000,-0.017052,0.000000,0.0,0.000000,-0.015306,0.000000,0.015338


In [3]:
sym_fea = pd.read_csv("../../../../extract_pubmed/2020_TF-IDF_dropdup.csv",index_col=0)
sym_fea[:2]

,Pediatric Obesity,Orthostatic Intolerance,Seizures,Muscle Weakness,Persistent Vegetative State,Chills,Sweating Sickness,Ataxia,Nocturia,Fetal Distress,...,Hypercalciuria,Chronic Pain,Hematemesis,Angina Pectoris,"Vision, Low",Muscle Hypertonia,"Hearing Loss, Functional",Breakthrough Pain,Mutism,Cerebrospinal Fluid Otorrhea
"Nevus, Intradermal",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Superinfection,0.0,0.0,0.0,0.635781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.309646,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
condis = set(screed_fea.index) & set([s.lower() for s in sym_fea.index])
condis = list(condis)
print(len(condis))
print(condis[:10])

54
['pre-eclampsia', 'nephrosclerosis', 'urticaria', 'lung injury', 'alpha 1-antitrypsin deficiency', 'dermatomyositis', 'liver cirrhosis', 'osteoporosis', 'adrenoleukodystrophy', 'intracranial aneurysm']


### Calculate the cosine/euc/spearman between 54 overlapping disease pairs and extract the corresponding model prediction results

In [32]:
predict_score = pd.read_csv("../../visualization/model_predictAll_result.csv",index_col=0)
predict_score[:2]

E:\Anaconda\envs\tf2\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,predict_score
0,0.911887
1,0.001816


In [33]:
#extract index of diseases
dis_index_inSym = []
sym_fea_index_lower = [s.lower() for s in sym_fea.index]

for dis in condis:
    index_inSym = sym_fea_index_lower.index(dis)
    dis_index_inSym.append(index_inSym)
    
print(len(dis_index_inSym))
dis_index_inSym[:5]

54


[619, 234, 3805, 3791, 4016]

In [19]:
def cosine_similarity(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    return num / (denom+0.000001)

from scipy.spatial import distance
import scipy.stats

In [34]:
scoreABs = []
scoreBAs = []
screed_cosine = [] 
screed_euc = [] 
pearson = [] 

for disA in condis:
    disA_index = dis_index_inSym[condis.index(disA)]
    
    for disB in condis:

        if disA==disB:
            continue
        disB_index = dis_index_inSym[condis.index(disB)]
        scoreAB = predict_score.loc[disA_index*4268+disB_index]
        scoreBA = predict_score.loc[disB_index*4268+disA_index]
        
        scoreABs.append(scoreAB)
        scoreBAs.append(scoreBA)
        
        cossim = 1-distance.cosine(screed_fea.loc[disA],screed_fea.loc[disB])
        screed_cosine.append(cossim)

        eucsim = distance.euclidean(screed_fea.loc[disA],screed_fea.loc[disB])
        screed_euc.append(eucsim)
        
        pearson.append(scipy.stats.pearsonr(screed_fea.loc[disA],screed_fea.loc[disB])[0])
        
print(len(screed_cosine))

E:\Anaconda\envs\tf2\lib\site-packages\scipy\spatial\distance.py:728: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
E:\Anaconda\envs\tf2\lib\site-packages\scipy\stats\stats.py:3913: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


2862


In [35]:

scores_predict=[] 
scores_predict = np.sum([scoreABs,scoreBAs],axis=0)/2
scores_predict = scores_predict.squeeze()

print(len(screed_cosine) == len(scores_predict))

True


### Compare distribution differences

In [36]:
screed_cosine_sim = []
screed_cosine_unsim = []

screed_euc_sim = []
screed_euc_unsim = []

screed_pearson_sim = []
screed_pearson_unsim = []

for i in range(len(scores_predict)):
    score = scores_predict[i]
    if score>=0.5:
        screed_cosine_sim.append(screed_cosine[i])
        screed_euc_sim.append(screed_euc[i])
        screed_pearson_sim.append(pearson[i])
    else:
        screed_cosine_unsim.append(screed_cosine[i])
        screed_euc_unsim.append(screed_euc[i])
        screed_pearson_unsim.append(pearson[i])
        
print(len(screed_euc_sim),"--",len(screed_euc_sim)==len(screed_cosine_sim))

688 -- True


In [25]:
# pd.DataFrame(screed_cosine_sim,columns=["cosine_similarity"]).to_csv("screed_cos_sim.csv")
# pd.DataFrame(screed_cosine_unsim,columns=["cosine_similarity"]).to_csv("screed_cos_unsim.csv")

# pd.DataFrame(screed_euc_sim,columns=["euc_similarity"]).to_csv("screed19717_euc_sim.csv")
# pd.DataFrame(screed_euc_unsim,columns=["euc_similarity"]).to_csv("screed19717_euc_unsim.csv")

pd.DataFrame(screed_pearson_sim,columns=["pearson_similarity"]).to_csv("screed19717_pearson_sim.csv")
pd.DataFrame(screed_pearson_unsim,columns=["pearson_similarity"]).to_csv("screed19717_pearson_unsim.csv")

In [37]:
print(screed_cosine_sim[:10])
print(screed_euc_sim[:10])
print(screed_pearson_sim[:10])

[-0.0516060595409793, -0.07483774351204309, 0.025150073234468828, 0.08622842929710417, 0.029926832210931065, -0.05476103373474328, 0.07148740105941198, 0.00899552100354395, -0.059266730808051005, nan]
[0.6418279503637585, 0.38440908440854116, 0.39220955554615067, 0.6154608280236288, 0.24795354349266166, 0.5122200223355504, 0.10769680246272742, 0.6200498737331294, 0.626405794670581, 0.09602887530758218]
[-0.11353877276580301, -0.08946159602608193, 0.008044280658398174, 0.0497288999799656, 0.007611334891150474, -0.08173937982271354, 0.0942208635892223, 0.01780554362353752, -0.06370320613181325, nan]


In [27]:

d1 = []
d2 = []
for x in screed_cosine_sim:
    if x <=1 and x>=-1:
        d1.append(x)
        
for x in screed_cosine_unsim:
    if x <=1 and x>=-1:
        d2.append(x)

In [38]:
import scipy.stats as stats
cosP = stats.mannwhitneyu(d1,d2,alternative='two-sided')
eucP = stats.mannwhitneyu(screed_euc_sim,screed_euc_unsim,alternative='two-sided')
pearsonP = stats.mannwhitneyu(screed_pearson_sim,screed_pearson_unsim,alternative='two-sided')
print(cosP,eucP,pearsonP)

MannwhitneyuResult(statistic=773648.0, pvalue=0.17214339254347522) MannwhitneyuResult(statistic=708328.0, pvalue=0.036398151831456296) MannwhitneyuResult(statistic=774198.0, pvalue=0.16318796766289678)


In [15]:
concat_screed = pd.DataFrame({"Score": d1+d2,"Methods": ["sim"]*len(d1) + ["unsim"]*len(d2)})

In [16]:
# concat_screed.to_csv("con_screed.csv")

### results
- screed_284
    - cos:MannwhitneyuResult(statistic=679376.0, pvalue=0.713110255584441) 
    - euc:MannwhitneyuResult(statistic=708328.0, pvalue=0.036398151831456296)
- screed_19717
    - cos:MannwhitneyuResult(statistic=773648.0, pvalue=0.17214339254347522) 
    - euc:MannwhitneyuResult(statistic=648640.0, pvalue=1.5034931453078025e-07)
    - pearson:MannwhitneyuResult(statistic=774544.0, pvalue=0.15773056868258228)

In [18]:
print(np.mean(screed_euc_sim))
print(np.mean(screed_euc_unsim))

1.104205160063662
1.1469193406487321


In [29]:
print(np.mean(screed_pearson_sim))
print(np.mean(screed_pearson_unsim))

0.013505160680879008
0.006062481685425086
